In [1]:
params = []
params.append(["train_size", "K", "learn_rate", "iterations", "w", "error"])

In [2]:
%%time
import numpy as np
import csv
import math

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'
profile_file = 'profiles.csv'

train_size = .9
K = 40
learn_rate = 0.0000001
iterations = 10
w = .1


#load CSV
train_data = {}
test_data = {}
ndata = 0
with open(train_file, 'r') as train_fh:
    ndata = sum(1 for row in train_fh) - 1
    
splitting = np.random.binomial(1, train_size, size=ndata)


Wall time: 1.95 s


In [3]:
%%time
artist_set = set()
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    
    for i, row, in enumerate(train_csv):
        user   = row[0]
        artist = row[1]
        artist_set.add(artist)
        plays  = row[2]

        if splitting[i]: 
            if not user in train_data:
                train_data[user] = {}
            train_data[user][artist] = int(plays)
            
        else:
            if not user in test_data:
                test_data[user] = {}

            test_data[user][artist] = int(plays)
              
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

Wall time: 33.8 s


In [4]:
assert(len(artist_set) == 2000)

In [5]:
%%time
with open(profile_file, 'r') as profile_fh:
    profile_csv = csv.reader(profile_fh, delimiter=',', quotechar='"')
    next(profile_csv, None)
    
    for i, row in enumerate(profile_csv):
        user = row[0]
        if row[1] == 'm':
            train_data[user]['m'] = 1
        if row[2] != '':
            train_data[user][int(row[2])] = 1
        if row[3] != '':
            train_data[user][row[3]] = 1

Wall time: 972 ms


In [6]:
%%time
user_feats = {}
artist_feats = {}


init = np.array(K * [math.sqrt(float(global_median)/K)])

artist_plays = {}
for user, user_data in train_data.iteritems():
    if not user in user_feats:
        user_feats[user] = init
    for artist, plays in user_data.iteritems():
        if not artist in artist_plays:
            artist_plays[artist] = []
        artist_plays[artist].append(plays)
        if not artist in artist_feats:
            artist_feats[artist] = init
 

Wall time: 4.16 s


In [7]:
%%time
artist_meds = {}
for artist in artist_plays.keys():
    artist_meds[artist] = np.median(np.array(artist_plays[artist]))

Wall time: 491 ms


In [ ]:
%%time
def predict_plays(user, artist):
    if (not user in user_feats) or (not artist in artist_feats):
        return global_median
    else:
        return np.dot(np.array(user_feats[user]), np.array(artist_feats[artist]))

    
def modify_prediction(user, artist, prediction):
    user_bias = global_median - user_medians[user]
    
    return w*prediction + (1-w)*user_medians[user]
    #return w*prediction + (1-w)*(artist_meds[artist] - user_bias)

def train(data):
    i = 0
    
    for k in range(iterations):
        for j, (user, user_data) in enumerate(train_data.iteritems()):
            if j % 1000 == 0:
                print "\r", j,k,

            for artist, plays in user_data.iteritems(): 
                
                pred = predict_plays(user,artist)
                error = int(plays) - pred

                artist_feats[artist] += learn_rate*error*user_feats[user]
                user_feats[user] +=  learn_rate*error*artist_feats[artist]  

train(train_data)

80000 6 

In [11]:
%%time
def calc_error(data):
    errors = []
    pred = 0
    for j, (user, user_data) in enumerate(data.iteritems()):
        if j % 1000 == 0:
            print "\r", j,
        for artist, plays in user_data.iteritems():
            if artist in artist_set:
                pred = predict_plays(user,artist)
                
                newpred = modify_prediction(user,artist,pred)
            
      
                error = abs(int(plays) - newpred)
                
                errors.append(error)
               
                
    return sum(np.array(errors))/len(errors)
                
error = calc_error(test_data)

print "Error: ", error


193000 Error:  141.760017489
Wall time: 4.11 s


[['train_size', 'K', 'learn_rate', 'iterations', 'w'], [0.7, 40, 1e-07, 7, 0.6]]


In [ ]:
params.append([train_size, K, learn_rate, iterations, w, error])